In [70]:
%load_ext autoreload
%autoreload 2
import os, sys
sys.path.append('..')
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context("poster")
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (11,8)
from merf.utils import MERFDataGenerator
from merf.merf import MERF

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Experimental Setup

There are some global parameters for all the experiments. Each experiment is run N_per_experiment times. The experiment itself is parametrized by three parameters of the generative model. We collect up the results of the experiments in a big list of dictinaries. This is then used to compute certain summary statistics after all the experiments are over.

In [82]:
# Globals
num_clusters_each_size = 20
train_sizes = [1, 3, 5, 7, 9]
known_sizes = [9, 27, 45, 63, 81]
new_sizes = [10, 30, 50, 70, 90]
n_estimators = 300
max_iterations = 20
train_cluster_sizes = MERFDataGenerator.create_cluster_sizes_array(train_sizes, num_clusters_each_size)
known_cluster_sizes = MERFDataGenerator.create_cluster_sizes_array(known_sizes, num_clusters_each_size)
new_cluster_sizes = MERFDataGenerator.create_cluster_sizes_array(new_sizes, num_clusters_each_size)

In [83]:
# Number of times to run each experiemnts 
N_per_experiment = 10

In [84]:
# Defining the experiments to run
experiments = [{'id': 0, 'm': .8, 'sigma_b_sq': 0.9, 'sigma_e': 1},
               {'id': 1, 'm': .7, 'sigma_b_sq': 2.7, 'sigma_e': 1},
               {'id': 2, 'm': .6, 'sigma_b_sq': 4.5, 'sigma_e': 1},
               {'id': 3, 'm': .3, 'sigma_b_sq': 0.2, 'sigma_e': 1},
               {'id': 4, 'm': .3, 'sigma_b_sq': 0.5, 'sigma_e': 1},
               {'id': 5, 'm': .2, 'sigma_b_sq': 0.8, 'sigma_e': 1}]

# Run Experiments

In [85]:
# Creating a dictionary to hold the results of the experiments
results = []
for experiment in experiments:
    results.append({'id': experiment['id'], 'ptev': [], 'prev': [],
                    'mse_known_rf_fixed': [], 'mse_known_rf_ohe': [], 'mse_known_merf': [], 
                    'mse_new_rf_fixed': [], 'mse_new_rf_ohe': [], 'mse_new_merf': []})
    
for experiment, result in zip(experiments, results): 
    for experiment_iteration in range(0, N_per_experiment):
        print("Experiment iteration: {}".format(experiment_iteration))
        # Generate data for experiment
        dgm = MERFDataGenerator(m=experiment['m'], sigma_b=np.sqrt(experiment['sigma_b_sq']), sigma_e=experiment['sigma_e'])
        train, test_known, test_new, train_cluster_ids, ptev, prev = dgm.generate_split_samples(train_cluster_sizes, known_cluster_sizes, new_cluster_sizes)
        
        # Store off PTEV and PREV
        result['ptev'].append(ptev)
        result['prev'].append(prev)
        
        # Training Data Extract
        X_train = train[['X_0', 'X_1', 'X_2']]
        Z_train = train[['Z']]
        clusters_train = train['cluster']
        y_train = train['y']

        # Known Cluster Data Extract
        X_known = test_known[['X_0', 'X_1', 'X_2']]
        Z_known = test_known[['Z']]
        clusters_known = test_known['cluster']
        y_known = test_known['y']

        # New Cluster Data Extract
        X_new = test_new[['X_0', 'X_1', 'X_2']]
        Z_new = test_new[['Z']]
        clusters_new = test_new['cluster']
        y_new = test_new['y']

        # MERF
        print("---------------------MERF----------------------")
        mrf = MERF(n_estimators=n_estimators, max_iterations=max_iterations)
        mrf.fit(X_train, Z_train, clusters_train, y_train)
        y_hat_known_merf = mrf.predict(X_known, Z_known, clusters_known)
        y_hat_new_merf = mrf.predict(X_new, Z_new, clusters_new)
        mse_known_merf = np.mean((y_known - y_hat_known_merf) ** 2)
        mse_new_merf = np.mean((y_new - y_hat_new_merf) ** 2)
        result['mse_known_merf'].append(mse_known_merf)
        result['mse_new_merf'].append(mse_new_merf)

        # Random Forest Fixed Only 
        print("---------------------Random Forest Fixed Effect Only----------------------")
        rf = RandomForestRegressor(n_estimators=n_estimators, n_jobs=-1)
        rf.fit(X_train, y_train)
        y_hat_known_rf_fixed = rf.predict(X_known)
        y_hat_new_rf_fixed = rf.predict(X_new)
        mse_known_rf_fixed = np.mean((y_known - y_hat_known_rf_fixed) ** 2)
        mse_new_rf_fixed = np.mean((y_new - y_hat_new_rf_fixed) ** 2)
        result['mse_known_rf_fixed'].append(mse_known_rf_fixed)
        result['mse_new_rf_fixed'].append(mse_new_rf_fixed)
        
        # Random Forest with OHE Cluster
        print("---------------------Random Forest w OHE Cluster----------------------")
        X_train_w_ohe = MERFDataGenerator.create_X_with_ohe_clusters(X_train, clusters_train, train_cluster_ids)
        X_known_w_ohe = MERFDataGenerator.create_X_with_ohe_clusters(X_known, clusters_known, train_cluster_ids)
        X_new_w_ohe = MERFDataGenerator.create_X_with_ohe_clusters(X_new, clusters_new, train_cluster_ids)
        rf_ohe = RandomForestRegressor(n_estimators=n_estimators, n_jobs=-1)
        rf_ohe.fit(X_train_w_ohe, y_train)
        y_hat_known_rf_ohe = rf_ohe.predict(X_known_w_ohe)
        y_hat_new_rf_ohe = rf_ohe.predict(X_new_w_ohe)
        mse_known_rf_ohe = np.mean((y_known - y_hat_known_rf_ohe) ** 2)
        mse_new_rf_ohe = np.mean((y_new - y_hat_new_rf_ohe) ** 2)
        result['mse_known_rf_ohe'].append(mse_known_rf_ohe)
        result['mse_new_rf_ohe'].append(mse_new_rf_ohe)


INFO     [utils.py:164] Drew 10000 samples from 200 clusters.
INFO     [utils.py:165] PTEV = 90.27098943243548, PREV = 9.69980451733243.


Experiment iteration: 0
---------------------MERF----------------------


INFO     [merf.py:235] GLL is 952.9395204510134 at iteration 1.
INFO     [merf.py:235] GLL is 915.3469052713147 at iteration 2.
INFO     [merf.py:235] GLL is 914.9601989823752 at iteration 3.
INFO     [merf.py:235] GLL is 911.7658909065897 at iteration 4.
INFO     [merf.py:235] GLL is 917.903395566948 at iteration 5.
INFO     [merf.py:235] GLL is 917.2753778606174 at iteration 6.
INFO     [merf.py:235] GLL is 913.0795994858006 at iteration 7.
INFO     [merf.py:235] GLL is 915.3712566406717 at iteration 8.
INFO     [merf.py:235] GLL is 926.8136354482265 at iteration 9.
INFO     [merf.py:235] GLL is 915.0300952588981 at iteration 10.
INFO     [merf.py:235] GLL is 910.2390676378507 at iteration 11.
INFO     [merf.py:235] GLL is 908.4455416486508 at iteration 12.
INFO     [merf.py:235] GLL is 912.3068560864838 at iteration 13.
INFO     [merf.py:235] GLL is 920.1124830602155 at iteration 14.
INFO     [merf.py:235] GLL is 917.2474943683274 at iteration 15.
INFO     [merf.py:235] GLL is 907.1

---------------------Random Forest Fixed Effect Only----------------------
---------------------Random Forest w OHE Cluster----------------------


/Users/souravdey/.virtualenvs/merf/lib/python3.6/site-packages/pandas/core/internals.py:490: RuntimeWarning: None of the categories were found in values. Did you mean to use
'Categorical.from_codes(codes, categories)'?
  return self.make_block(Categorical(self.values, **kwargs))
INFO     [utils.py:164] Drew 10000 samples from 200 clusters.
INFO     [utils.py:165] PTEV = 89.5974778548728, PREV = 10.449256111627585.


...

/Users/souravdey/.virtualenvs/merf/lib/python3.6/site-packages/pandas/core/internals.py:490: RuntimeWarning: None of the categories were found in values. Did you mean to use
'Categorical.from_codes(codes, categories)'?
  return self.make_block(Categorical(self.values, **kwargs))
INFO     [utils.py:164] Drew 10000 samples from 200 clusters.
INFO     [utils.py:165] PTEV = 56.770349019949116, PREV = 60.9186333730095.


Experiment iteration: 9
---------------------MERF----------------------

INFO     [merf.py:235] GLL is 644.0301027331609 at iteration 1.
INFO     [merf.py:235] GLL is 557.7380533998236 at iteration 2.
INFO     [merf.py:235] GLL is 564.2565396773457 at iteration 3.
INFO     [merf.py:235] GLL is 557.3810511689695 at iteration 4.
INFO     [merf.py:235] GLL is 565.9282817812142 at iteration 5.
INFO     [merf.py:235] GLL is 563.0694156819429 at iteration 6.
INFO     [merf.py:235] GLL is 565.4937912592842 at iteration 7.
INFO     [merf.py:235] GLL is 563.0859056973304 at iteration 8.
INFO     [merf.py:235] GLL is 561.6207967029027 at iteration 9.
INFO     [merf.py:235] GLL is 556.7559587505148 at iteration 10.
INFO     [merf.py:235] GLL is 565.6854395687658 at iteration 11.
INFO     [merf.py:235] GLL is 555.7407926520369 at iteration 12.
INFO     [merf.py:235] GLL is 563.852482806235 at iteration 13.
INFO     [merf.py:235] GLL is 563.2157111793713 at iteration 14.
INFO     [merf.py:235] GLL is 560.6367506161175 at iteration 15.
INFO     [merf.py:235] GLL is 566.7


---------------------Random Forest Fixed Effect Only----------------------
---------------------Random Forest w OHE Cluster----------------------


/Users/souravdey/.virtualenvs/merf/lib/python3.6/site-packages/pandas/core/internals.py:490: RuntimeWarning: None of the categories were found in values. Did you mean to use
'Categorical.from_codes(codes, categories)'?
  return self.make_block(Categorical(self.values, **kwargs))


In [86]:
results

[{'id': 0,
  'mse_known_merf': [2.760523177093043,
   2.5087553696669889,
   2.5943145052516368,
   2.6191382025895829,
   2.9915420919390248,
   2.5731980513549435,
   2.5492818836814255,
   2.6178646075415268,
   2.6452967979724291,
   2.6349716403447956],
  'mse_known_rf_fixed': [3.4578330448297345,
   3.3030197425003642,
   3.1439550999880095,
   3.6209615738420875,
   3.6039473582364483,
   3.3775328152199404,
   3.3194500458977818,
   3.1927294475839054,
   3.265965109922873,
   3.1003040609178845],
  'mse_known_rf_ohe': [3.4830953017994477,
   3.2239581083475639,
   3.110565713929454,
   3.6166597842930543,
   3.7812827130356181,
   3.2338646071131398,
   3.4317161000726575,
   3.2487270792306147,
   3.2894341539626684,
   3.1240517007738293],
  'mse_new_merf': [3.5193318868215693,
   3.3605649178449739,
   3.1739032126075202,
   3.2976605094331286,
   3.5475359148960401,
   3.3155429286244149,
   3.9087842279383027,
   3.1593715707432319,
   3.1738679959817135,
   3.14974581672

In [87]:
import pickle
pickle.dump(results, open("results.pkl", "wb" ))

In [92]:
100 * np.mean((np.array(results[0]['mse_known_rf_fixed']) - np.array(results[0]['mse_known_merf'])) / np.array(results[0]['mse_known_rf_fixed']))

20.538988592302683

In [93]:
def merf_gain(merf_mse, non_merf_mse):
    return 100 * np.mean((np.array(non_merf_mse) - np.array(merf_mse)) / np.array(non_merf_mse))

In [105]:
summary_results = pd.DataFrame(np.zeros((len(results), 6)), columns=['ptev', 'prev', 'known merf-rf fixed gain', 'new merf-rf fixed gain', 'known merf-rf ohe gain', 'new merf-rf ohe gain'])
for result in results: 
    summary_results.loc[result['id'], 'ptev'] = np.mean(np.array(result['ptev']))
    summary_results.loc[result['id'], 'prev'] = np.mean(np.array(result['prev']))
    summary_results.loc[result['id'], 'known merf-rf fixed gain'] = merf_gain(result['mse_known_merf'], result['mse_known_rf_fixed'])
    summary_results.loc[result['id'], 'new merf-rf fixed gain']= merf_gain(result['mse_new_merf'], result['mse_new_rf_fixed'])
    summary_results.loc[result['id'], 'known merf-rf ohe gain'] = merf_gain(result['mse_known_merf'], result['mse_known_rf_ohe'])
    summary_results.loc[result['id'], 'new merf-rf ohe gain'] = merf_gain(result['mse_new_merf'], result['mse_new_rf_ohe'])

In [106]:
summary_results

,ptev,prev,known merf-rf fixed gain,new merf-rf fixed gain,known merf-rf ohe gain,new merf-rf ohe gain
0,89.870057,10.145162,20.538989,-0.186749,20.879242,3.251213
1,89.771511,30.764805,51.514512,5.267747,42.852774,3.936174
2,90.016328,49.909787,68.000664,8.080870,55.684546,4.857849
3,56.821840,15.200795,6.205787,0.773231,2.662849,-1.341002
4,61.926701,30.743338,20.877254,3.143301,12.649553,0.679079
5,56.508651,61.573972,32.931911,4.541009,14.726535,0.728173
